# Expanding data for Open Rice restaurants via JSON API calls and visualization - PART 2

<img src="data/openrice.png">

This analysis starts with open data from Open Rice - Hong Kong's most popular dining guide to help people find places to eat based on the restaurant reviews written by real local people. In part 1 of our data analysis, we used that data from Open Rice and API calls to get latitude and longitude information for each restaurant. In part 2, we will use our JSON data to do some analyses and data visualizations.



### Parsing JSON

In our previous notebook we got the longitude and latitude for all of the resturants in the open rice dataset and exported out the JSON data, however this is still in the JSON format. We will need to import that JSON data and then extract the longitude and latitude data.

In [1]:
# loading our original CSV data as a pandas dataframe
import pandas as pd

df = pd.read_csv('data/open-rice.csv')
df.head()

,address,bookmarks,dislikes,food_type,likes,name,number_of_reviews,price_range
0,"Shop J-K., 200 Hollywood Road,",5838,6,Hong Kong Style,78,For Kee Restaurant 科記咖啡餐室,(133 Reviews),Below $50
1,"G/F, 108 Hollywood Road,",3492,2,International,20,Blue · Butcher & Meat Specialist,(30 Reviews),$201-400
2,"G/F, 206 Hollywood Road,",5517,5,Thai,31,Chachawan,(43 Reviews),$201-400
3,"Shop 3018, 3/F, Shun Tak Centre, 168-200 Conna...",1173,1,Hong Kong Style,23,Capital Café 華星冰室,(39 Reviews),Below $50
4,"G/F, 38 Queens Road West,",1064,1,Indian,50,Namaste Kitchen 滋味廚房,(57 Reviews),$51-100


In [2]:
import json

In [3]:
# loading our json data a list
addresses = json.load(open("data/openrice_addresses.json"))

In [4]:
# checking out our JSON data
print(addresses[5])

{"RequestAddress": {"AddressLine": ["Shop 1, G/F, King Ho Building, 41-49 Aberdeen Street, Soho,"]}, "SuggestedAddress": [{"Address": {"PremisesAddress": {"EngPremisesAddress": {"BuildingName": "KING HO BUILDING", "EngStreet": {"StreetName": "ABERDEEN STREET", "BuildingNoFrom": "41", "BuildingNoTo": "49"}, "EngDistrict": {"DcDistrict": "CW"}, "Region": "HK"}, "ChiPremisesAddress": {"Region": "\u9999\u6e2f", "ChiDistrict": {"DcDistrict": "CW"}, "ChiStreet": {"StreetName": "\u9d28\u5df4\u7538\u8857", "BuildingNoFrom": "41", "BuildingNoTo": "49"}, "BuildingName": "\u91d1\u8c6a\u5927\u5ec8"}, "GeospatialInformation": [{"Northing": "815981", "Easting": "833655", "Latitude": "22.2827", "Longitude": "114.1515"}, {"Northing": "815979", "Easting": "833655", "Latitude": "22.2826", "Longitude": "114.1515"}]}}, "ValidationInformation": {"ValidationTime": null}}]}


Our data is a list of JSON strings. We'll need to define a function that extracts each one of the latitude and longitudes from each JSON string.

In [5]:
# if we take a look at a single JSON line after parsing it into a python dictionary
# we can see a format so we can extract what we need (which is the request address and the lat and long)
json.loads(addresses[5])

{'RequestAddress': {'AddressLine': ['Shop 1, G/F, King Ho Building, 41-49 Aberdeen Street, Soho,']},
 'SuggestedAddress': [{'Address': {'PremisesAddress': {'EngPremisesAddress': {'BuildingName': 'KING HO BUILDING',
      'EngStreet': {'StreetName': 'ABERDEEN STREET',
       'BuildingNoFrom': '41',
       'BuildingNoTo': '49'},
      'EngDistrict': {'DcDistrict': 'CW'},
      'Region': 'HK'},
     'ChiPremisesAddress': {'Region': '香港',
      'ChiDistrict': {'DcDistrict': 'CW'},
      'ChiStreet': {'StreetName': '鴨巴甸街',
       'BuildingNoFrom': '41',
       'BuildingNoTo': '49'},
      'BuildingName': '金豪大廈'},
     'GeospatialInformation': [{'Northing': '815981',
       'Easting': '833655',
       'Latitude': '22.2827',
       'Longitude': '114.1515'},
      {'Northing': '815979',
       'Easting': '833655',
       'Latitude': '22.2826',
       'Longitude': '114.1515'}]}},
   'ValidationInformation': {'ValidationTime': None}}]}

In [6]:
# Getting a test address we requested from the original CSV file
json.loads(addresses[5])['RequestAddress']['AddressLine'][0]

'Shop 1, G/F, King Ho Building, 41-49 Aberdeen Street, Soho,'

In [7]:
# Getting a test latitude from the nested dictionary:
json.loads(addresses[5])['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Latitude']

'22.2827'

In [8]:
# Getting a test longitude from the nested dictionary:
json.loads(addresses[5])['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Longitude']

'114.1515'

In [9]:
len(addresses)

14693

In [10]:
import numpy as np

# creating a dictionary of addresses as keys and latitude and longitude as a tuple of two ints:
addlatlong = {}

for i in range(len(addresses)):
    address = json.loads(addresses[i])['RequestAddress']['AddressLine'][0]
    try:
        latitude = float(json.loads(addresses[i])['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Latitude'])
        longitude = float(json.loads(addresses[i])['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation'][0]['Longitude'])
    except:
        latitude, longitude = np.nan, np.nan
    addlatlong[address] = [latitude, longitude]

# printing out a sample to show it worked:
print(list(addlatlong.keys())[0:5])
print()
print(list(addlatlong.values())[0:5])

['Shop J-K., 200 Hollywood Road,', 'G/F, 108 Hollywood Road,', 'G/F, 206 Hollywood Road,', 'Shop 3018, 3/F, Shun Tak Centre, 168-200 Connaught Road Central,', 'G/F, 38 Queens Road West,']

[[22.2852, 114.1478], [22.2835, 114.1511], [22.2853, 114.1477], [22.2879, 114.1518], [22.2867, 114.1479]]


### Updating our Original Dataframe and Adding a Latitude and Longitude Column to each Address

In [11]:
# converting our address, lat, long dictionary to a pandas dataframe:
addlatlong_df = pd.DataFrame.from_dict(addlatlong, orient='index', columns=['latitude', 'longitude'])

# add address as a column so we can use it to merge later
addlatlong_df['address'] = addlatlong_df.index
addlatlong_df.head()

,latitude,longitude,address
"Shop J-K., 200 Hollywood Road,",22.2852,114.1478,"Shop J-K., 200 Hollywood Road,"
"G/F, 108 Hollywood Road,",22.2835,114.1511,"G/F, 108 Hollywood Road,"
"G/F, 206 Hollywood Road,",22.2853,114.1477,"G/F, 206 Hollywood Road,"
"Shop 3018, 3/F, Shun Tak Centre, 168-200 Connaught Road Central,",22.2879,114.1518,"Shop 3018, 3/F, Shun Tak Centre, 168-200 Conna..."
"G/F, 38 Queens Road West,",22.2867,114.1479,"G/F, 38 Queens Road West,"


In [13]:
# creating our final df by merging the above information together
openrice = df.merge(addlatlong_df, left_on='address', right_on='address')

openrice.head()

,address,bookmarks,dislikes,food_type,likes,name,number_of_reviews,price_range,latitude,longitude
0,"Shop J-K., 200 Hollywood Road,",5838,6,Hong Kong Style,78,For Kee Restaurant 科記咖啡餐室,(133 Reviews),Below $50,22.2852,114.1478
1,"Shop J-K., 200 Hollywood Road,",5838,6,Tea Restaurant,78,For Kee Restaurant 科記咖啡餐室,(133 Reviews),Below $50,22.2852,114.1478
2,"G/F, 108 Hollywood Road,",3492,2,International,20,Blue · Butcher & Meat Specialist,(30 Reviews),$201-400,22.2835,114.1511
3,"G/F, 108 Hollywood Road,",2259,10,Western,35,Classified,(71 Reviews),$101-200,22.2835,114.1511
4,"G/F, 108 Hollywood Road,",3492,2,International,20,Blue · Butcher & Meat Specialist,(30 Reviews),$201-400,22.2835,114.1511


Let's export this new data to a CSV file:

In [15]:
openrice.to_csv(path_or_buf="data/open-rice_geotagged.csv", index=False)

## Visualizing our New Data